# Agenda: Week 2

1. Recap
2. Dtypes (the types of data that we can use in Pandas)
3. `NaN` ("not a number")
4. DataFrames (2D data structures that we use in Pandas)
5. Adding and removing data in our data frames
6. Useful methods in our data frames
7. Querying with boolean indexes
8. Querying with `loc`
9. Reading CSV data (meaning: Real-world data into our data frames)

# Last week, quick recap

1. Pandas has two types of data
    - series (1D)
    - data frame (2D)
2. We can create a series by passing a Python list (or a similar iterable, such as a NumPy array)
3. We can assign an index of our choosing to our series
    - By default, it numbers the elements starting at 0, just like a list/string/tuple
    - If we assign our own index, we can use integers, floats, or even strings
4. Retrieving
    - Retrieve via the index using `.loc`
    - Retrieve via the numeric position (starting at 0) with `.iloc`
    - We can use slices
5. Broadcasting
    - If we apply an operator to a series and a single value, that value is "broadcast" with the operator to every element of the series.  We get back a new series, the result of applying the value with the operator.
    - If we use broadcasting with `==` or other comparison operators, we get back a "boolean series," containing only `True` and `False` values
6. Boolean indexing
    - If we apply a boolean index to an existing series, then only those items that match a `True` value are returned.  Those matching a `False` value are ignored.
    - This is why we don't need to use `if` or `for` in Pandas.  We use boolean indexes to find all of the values that match what we want.

In [1]:
import numpy as np   # not critical, but might come in handy
import pandas as pd  # uses the standard alias "pd"
from pandas import Series, DataFrame  # this lets me use the names without pd. before them

In [3]:
s = Series([10, 20, 30, 20, 30, 40, 20, 30, 40, 50]) # default, numeric index
s

0    10
1    20
2    30
3    20
4    30
5    40
6    20
7    30
8    40
9    50
dtype: int64

In [4]:
s.mean()  # what is the mean of our series?

29.0

In [6]:
s.sum() / s.count()

29.0

In [ ]:
# to retrieve an item, 